### Importing the required libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from iprocessor import add_day_name_column, add_date_name_column, smooth_sundays_rolling_w7_l

## loading the csv file from RKI

In [13]:
# Dataframe-------------------------------------------------
df = pd.read_csv(r'data_csv\German_case.csv')

#Convert 'Date' column to datetime format----------------------------------------------------------------------
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Modification------------------------------------------------------------------------------
# Add the 'days' column
df = add_day_name_column(df)
df = add_date_name_column(df)
# second modification with w7_l-----------------------------------------------------------------------------
df_observed = smooth_sundays_rolling_w7_l(df)
# -----------------------------------------------------------------------------------------------------
# Taking 'days' time column from dataframe
t_fit_base = np.array(df_observed['days'])
tmax_base = len(t_fit_base)
#time
t_fit_0 = np.zeros((tmax_base,2))
t_fit_0[:,0] = t_fit_base
t_fit_0[:,1] = 0
#
t_fit_1 = np.zeros((tmax_base,2))
t_fit_1[:,0] = t_fit_base
t_fit_1[:,1] = 1
#
t_fit = np.r_[t_fit_0,t_fit_1]
len(t_fit)
#
tmax = len(t_fit)

## Setting the initial values and parameters for the SEAIFRD model

In [14]:
# Sample parameters,
contacts = 2.0
transmission_prob = 0.3649
total_population = 84000000
reducing_transmission = 0.764
exposed_period = 5.2  #
asymptomatic_period = 7
infectious_period = 3.7
isolated_period = 11  # 11,23
prob_asymptomatic = 0.2
prob_quarant_inf = 0.05
test_asy = 0.171
dev_symp = 0.125
mortality_isolated = 0.002
mortality_infected = 0.01

# Sample initial conditions-------------------------------------------------------
total_population = 82000000  # Total number of individuals 2020 may
E0 = 18633.333333333332
A0 = 5064
I0 = 136.89
F0 = 1451
R0 = 2065
D0 = 47
S0 = total_population - E0 - A0 - I0 - F0 - R0 - D0
initial_conditions = [S0, E0, A0, I0, F0, R0, D0]

## Derivative Function for the Model

In [15]:
def derivative_rhs(t, X, contacts, transmission_prob, total_population, reducing_transmission,
                   exposed_period, asymptomatic_period, infectious_period, isolated_period,
                   prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected):
    S, E, A, I, F, R, D = X
    derivS = - contacts * transmission_prob * S * (I + reducing_transmission * A) / total_population
    derivE = contacts * transmission_prob * S * (I + reducing_transmission * A) / total_population - E / exposed_period
    derivA = prob_asymptomatic * E / exposed_period - A / asymptomatic_period
    derivI = (
                         1 - prob_asymptomatic) * E / exposed_period + dev_symp * A / asymptomatic_period - I / infectious_period  # +
    derivF = prob_quarant_inf * I / infectious_period - F / isolated_period + test_asy * A / asymptomatic_period  # prob_isolated_asy*A/asymptomatic_period
    derivR = (1 - prob_quarant_inf - mortality_infected) * I / infectious_period + (
                1 - mortality_isolated) * F / isolated_period + (
                         1 - dev_symp - test_asy) * A / asymptomatic_period  # (1-prob_isolated_asy)*A / asymptomatic_period
    derivD = (mortality_infected) * I / infectious_period + mortality_isolated * F / isolated_period
    return np.array([derivS, derivE, derivA, derivI, derivF, derivR, derivD])


In [16]:
def seaifrd_model(t, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                  exposed_period, asymptomatic_period, infectious_period, isolated_period,
                  prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected):
    def derivative(t, initial_conditions):
        return derivative_rhs(t, initial_conditions, contacts, transmission_prob, total_population,
                              reducing_transmission,
                              exposed_period, asymptomatic_period, infectious_period,
                              isolated_period, prob_asymptomatic,
                              prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)

    solution = solve_ivp(derivative, [0, tmax], initial_conditions, t_eval=t, method='RK45')
    print(solution)
    return solution  # .y.flatten()

## Objective Function

In [17]:
def objective(t, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
              exposed_period, asymptomatic_period, infectious_period, isolated_period,
              prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected):
    temp = seaifrd_model(t, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                         exposed_period, asymptomatic_period, infectious_period,
                         isolated_period, prob_asymptomatic,
                         prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)
    # print(f' the name of the columns is: {type(temp)}')
    return temp  # [temp.y[5],temp.y[6]]


#----------------------------------------------------------------------------------------------------------------
# df_observed/ dataframe that have the cumulative as well as daily observed data(n_) for dead and recovered
# here we will combine the recovered and dead in one matrix with 2d
#['Date', 'Confirmed', 'Deaths', 'Recovered', 'n_confirmed', 'n_death','n_recovered', 'Infection_case', 'date_name', 'days', 'rolling_mean_r','rolling_mean_c', 'rolling_mean_d']
"""
data_recovered = np.zeros((df_observed['n_recovered'].size,2))
data_recovered[:,0] = df_observed['n_recovered']
#
data_dead = np.ones((df_observed['n_death'].size,2))
data_dead[:,0] = df_observed['n_death']
#recovered_dead = np.r_[data_recovered,data_dead]
recovered_dead = np.r_[data_dead,data_recovered]
"""
recovered_dead = np.concatenate((df_observed['n_death'],df_observed['n_recovered']))

#-----------------------------------------------------------------------------------------------------------------
def objective_function_recoverd_dead(X,isolated_period):
    t = X[:, 0]
    mode = X[:, 1]

    # Model parameters (could be passed as arguments or fixed here)
    contacts = 2
    transmission_prob = 0.3
    total_population = 82000000
    reducing_transmission = 0.55
    exposed_period = 5.2
    asymptomatic_period = 7
    infectious_period = 3.7
    #isolated_period = 12
    prob_asymptomatic = 0.34
    prob_quarant_inf = 0.9303
    test_asy = 0.271
    dev_symp = 0.125
    mortality_isolated = 0.02
    mortality_infected = 0.1

    # Solve the model
    t = t[:len(t)//2] #sorted(set(t))
    solution = seaifrd_model(t, contacts, initial_conditions, transmission_prob, total_population, reducing_transmission,
                             exposed_period, asymptomatic_period, infectious_period, isolated_period,
                             prob_asymptomatic, prob_quarant_inf, test_asy, dev_symp, mortality_isolated, mortality_infected)

    print(solution)
    recovered = solution.y[5]
    recovered = np.concatenate((recovered,recovered))
    dead = solution.y[6]
    dead = np.concatenate((dead,dead))
    retval = np.where(mode, recovered, dead)

    return retval


## Calling Curve_fit Function

In [18]:
t_end = df_observed['days'].iloc[-1]
# Create a sequence from 0 to t_end
#t_fit = np.arange(0, 2*tmax, 1)

popt, pcov = curve_fit(objective_function_recoverd_dead, t_fit, recovered_dead)


  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.482e+00  2.283e+00]
            ...
            [ 3.568e+03  4.909e+03 ...  6.527e+07  6.527e+07]
            [ 1.015e+02  2.002e+02 ...  6.075e+06  6.075e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 1214
     njev: 0
      nlu: 0
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.482e+00  2.283e+00]
            ...
            [ 3.568e+03  4.909e+03 ...  6.527e+07  6.527e+07]
            [ 1.015e+02  2.002e+02 ...  6.075e+06  6.075e+06]]
      sol: None
 t_events: None
 y_event

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.513e+03  2.979e+03 ...  2.179e+07  2.183e+07]
            [ 7.994e+01  1.608e+02 ...  5.188e+06  5.189e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 440
     njev: 0
      nlu: 0
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.513e+03  2.979e+03 ...  2.179e+07  2.183e+07]
            [ 7.994e+01  1.608e+02 ...  5.188e+06  5.189e+06]]
      sol: None
 t_events: None
 y_events

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.359e+07  1.359e+07]
            [ 7.991e+01  1.607e+02 ...  5.020e+06  5.021e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 440
     njev: 0
      nlu: 0
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.359e+07  1.359e+07]
            [ 7.991e+01  1.607e+02 ...  5.020e+06  5.021e+06]]
      sol: None
 t_events: None
 y_events

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.315e+07  1.315e+07]
            [ 7.991e+01  1.607e+02 ...  5.012e+06  5.012e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 440
     njev: 0
      nlu: 0
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.315e+07  1.315e+07]
            [ 7.991e+01  1.607e+02 ...  5.012e+06  5.012e+06]]
      sol: None
 t_events: None
 y_events

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.314e+07  1.314e+07]
            [ 7.991e+01  1.607e+02 ...  5.011e+06  5.011e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 440
     njev: 0
      nlu: 0
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.314e+07  1.314e+07]
            [ 7.991e+01  1.607e+02 ...  5.011e+06  5.011e+06]]
      sol: None
 t_events: None
 y_events

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.314e+07  1.314e+07]
            [ 7.991e+01  1.607e+02 ...  5.011e+06  5.011e+06]]
      sol: None
 t_events: None
 y_events: None
     nfev: 440
     njev: 0
      nlu: 0
  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 1.000e+00  2.000e+00 ...  2.680e+02  2.690e+02]
        y: [[ 8.197e+07  8.197e+07 ...  1.066e+07  1.066e+07]
            [ 1.764e+04  1.790e+04 ...  2.490e+00  2.290e+00]
            ...
            [ 2.511e+03  2.975e+03 ...  1.314e+07  1.314e+07]
            [ 7.991e+01  1.607e+02 ...  5.011e+06  5.011e+06]]
      sol: None
 t_events: None
 y_events

In [19]:
popt

array([1.58256794e+10])

## Visualization and curve_fit_error estimating

In [20]:
perr_r_d = np.sqrt(np.diag(pcov))
perr_r_d

array([1.8650141e+16])

In [ ]:
# perr
# cov and parameter error estimates is derived based on a linear approximation to the model function around the optimum [1]
'''@article{vugrin2007confidence,
  title={Confidence region estimation techniques for nonlinear regression in groundwater flow: Three case studies},
  author={Vugrin, Kay White and Swiler, Laura Painton and Roberts, Randall M and Stucky-Mack, Nicholas J and Sullivan, Sean P},
  journal={Water Resources Research},
  volume={43},
  number={3},
  year={2007},
  publisher={Wiley Online Library}
}'''
perr = np.sqrt(np.diag(pcov))
perr
# what does high and low value from perr mean: 

In [ ]:
# visualization
plt.plot(t_fit, func(tfit, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))